## PDF发票自动检测检测端解决方案（Version1.0）


#### JAKA Intern - Racheus Zhao
2024.7.2-2024.7.4

本周我的主要任务是做一个基于PDF格式发票的检测+提取功能的demo。经过三天左右的调试和修改，现做出第一个版本，满足测试的基础需要，后续根据甲方的需求以及汪博的意见继续改进。代码基于`Python`的`pdfplumber`库,通过面向对象的方式进行编写。

#### 综述

目前主要实现以下类型的发票信息：
* 电子发票(普通发票)
* 电子发票（增值税专用发票）
* (地区)增值税电子普通发票

在实际的使用过程中，用户并不需要知道数据清洗的底层逻辑，也不需要手动选择发票的类型。我们想打造这样一款自动识别发票类型的方案并以此为依据对数据进行清理。

根据面向对象的方法，我构建了三个类，通过调用类的封装和函数实现相关功能

* `Extractor`类：基础的检测函数，基于一个PDF文件，提取出相关信息，返回一个`Pd.DataFrame`格式的数据。

* `DataCleaner`类：通过传入文件路径为初始化方法，自动识别并调取清洗函数，返回一个列向量，包含发票的信息。
  
* `InvoiceDetector`类：集成上述两个功能的大类，可以输出xlsx文件等，后续可以考虑接入数据库，实现更加完善的系统功能。

#### `InvoiceDetector`类

作为用户，只需要知道`InvoiceDetector`类即可——
* `InvoiceDetector.Detect()`检测函数：

  自动实现文件夹中PDF发票文件的遍历，如果文件合法且可以被检测，将会在这个类的`InvoiceList`中添加该发票的信息，如果不能被检测（由于技术原因或其他，测试概率3%左右），将会把文件夹的名字记录在`InvoiceDetector.ErrorList`中，用户可选择调用来进行未读取文档的手动录入。

* `InvoiceDetector.Clean()`清理函数：

  主要是将每个单元格中还存在的冗余文字删除，这可能是由于发票供应商在开票时的习惯不同，不可能拥有一套完美的方案解决每种发票习惯导致的。
  
* `InvoiceDetector.Save(self,dest)`保存函数：
* 
  可以将这个文件夹中*所有能够读取的发票数据*归并输出成一个Excel表格，丰富系统功能。

这里给一个Demo：

_运行之前请确认_`pdfplumber`和`pandas`正确安装在当前Python环境下

In [1]:
from InvoiceDetector import InvoiceDetector
import os
import re
import pandas as pd
import pdfplumber as pb
import time

我将发票放在当前文件夹下的`invoice_2`文件夹中，将此作为路径，初始化一个`InvoiceDetector`类,并且引入时间函数来检测代码运行时间：

In [2]:
start_time = time.time()
path = '../invoice'
detector = InvoiceDetector(path)
detector.Detect()
detector.Clean()
detector.Check_Error_List()
end_time = time.time()

1
3
该发票类型为==电子发票（普通发票)==
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
==数据清洗完成==
==1_pdfDownloads.pdf== have been loaded
1
3
该发票类型为==电子发票（普通发票)==
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
==数据清洗完成==
==1_打印机维修.pdf== have been loaded
1
3
该发票类型为==电子发票（普通发票)==
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
==数据清洗完成==
==1_锁.pdf== have been loaded
1
3
该发票类型为==电子发票（普通发票)==
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
==数据清洗完成==
==24512000000047840269.pdf== have been loaded
1
3
该发票类型为==电子发票（普通发票)==
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
Repeated opeartor cleaned once.
==数据清洗完成==
==3_pdfDownloads.pdf== have been

In [3]:
execution_time = end_time - start_time
print("All Done!")
detector.Save('InvoiceList.xlsx')
print(f"发票检测程序运行时间: {execution_time} 秒")
print("The undetected files are :")
detector.Check_Error_List()

All Done!
Data Saved! -- path: InvoiceList.xlsx
发票检测程序运行时间: 3.022951364517212 秒
The undetected files are :
Error List: ['dzfp_24612000000048851623_都江堰拉法基水泥有限公司_20240531165902.pdf']


In [7]:
detector.InvoiceList

,发票号码,开票日期,购买方纳税人识别号,购买方名称,销售方纳税人识别号,销售方名称,价税合计(大写),价税合计(小写),备注,销售方开户行及账号,...,数量,单价,金额,税率,税额,销售方地址电话,收款人,复核,开票人,销售方
Invoice1,24512000000040156230,2024年03月13日,92510181MA7K43C125,都江堰市李红英蔬菜经营部,91510181MA61R1A37X,都江堰市锐利市场经营管理有限公司,玖仟贰佰圆整,¥9200.00,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Invoice2,23312000000073740052,2023年09月08日,9131000039865525XN,节卡机器人股份有限公司,91310114332509356M,上海印威办公设备有限公司,陆佰伍拾圆整,¥650.00,,华夏银行股份有限公司上海四平支行10577000000009939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Invoice3,23312000000060505387,2023年08月21日,9131000039865525XN,节卡机器人股份有限公司,913101175947330915,上海展涵装饰材料有限公司,贰拾贰圆捌角整,¥22.80,,中国银行股份有限公司上海市北京东路支行452062099211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Invoice4,24512000000047840269,2024年03月23日,91510181MA61R1A37X,都江堰市锐利市场经营管理有限公司,92510181MA6C9QY80J,都江堰市拾月闷骚鸡风味餐馆,玖佰贰拾壹圆整,¥921.00,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Invoice5,24512000000110890207,2024年06月07日,55510181572299159N,都江堰市幸福街道彩虹社区居民委员会,91510181MA61R1A37X,都江堰市锐利市场经营管理有限公司,柒仟贰佰圆整,¥7200.00,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Invoice6,24512000000131146892,2024年07月01日,91510181710920477L,都江堰拉法基水泥有限公司,91510182684598487M,彭州安先达运业有限公司,伍佰贰拾柒万陆仟柒佰肆拾捌圆贰角柒分,¥5276748.27,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Invoice7,24512000000131145453,2024年07月01日,91510181710920477L,都江堰拉法基水泥有限公司,91510182684598487M,彭州安先达运业有限公司,陆佰贰拾柒万壹仟伍佰伍拾贰圆柒角肆分,¥6271552.74,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Invoice8,23312000000088740764,2023年09月26日,123206214679976687,海安上海交通大学智能装备研究院,91310114564800185A,上海泰迪贸易有限公司,贰佰肆拾圆整,¥240.00,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Invoice9,23512000000114030712,2023年10月31日,91510181MA6A6HAK3R,成都市每日都江堰文化传播有限公司,51510181MJQ0713747,都江堰市篮球协会,肆万捌仟伍佰圆整,¥48500.00,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Invoice10,24512000000117504611,2024年06月17日,91510000078854751H,四川盛世飞跃通信有限公司,91510181064314754K,成都森睿电子科技有限公司都江堰分公司,壹仟陆佰捌拾圆整,¥1680.00,购买方地址:-;电话:-;购方开户银行:-;银行账号:-;销售方地址:四川省成都市都江堰市迎...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
type(detector.InvoiceList)

pandas.core.frame.DataFrame

In [6]:
type(detector.InvoiceList.iloc[0]['发票号码'])

str

可以看到大致上，完成100条数据的采集用时24s左右，当然这和PC的运算能力和水平有关。